In [1]:
#! /usr/bin/env python
# -- coding:utf-8 --

import cv2
import numpy as np
from matplotlib import pyplot as plt
import time

# Para usar o vídeo
#cap = cv2.VideoCapture('hall_box_battery_mp2.mp4')

# As 3 próximas linhas são para usar a webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)


def identifica_cor(frame):
    '''
    Segmenta o maior objeto cuja cor é parecida com cor_h (HUE da cor, no espaço HSV).
    '''

    # No OpenCV, o canal H vai de 0 até 179, logo cores similares ao 
    # vermelho puro (H=0) estão entre H=-8 e H=8. 
    # Veja se este intervalo de cores está bom
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    cor_menor = np.array([0, 150, 100])
    cor_maior = np.array([20, 255, 255])
    segmentado_cor = cv2.inRange(frame_hsv, cor_menor, cor_maior)

    # Será possível limpar a imagem segmentado_cor? 
    # Pesquise: https://docs.opencv.org/trunk/d9/d61/tutorial_py_morphological_ops.html
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
    
    #segmentado_cor = cv2.morphologyEx(segmentado_cor,cv2.MORPH_OPEN, kernel)
    segmentado_cor = cv2.morphologyEx(segmentado_cor, cv2.MORPH_CLOSE, kernel)
    #frame_hsv = cv2.erode(frame_hsv,kernel,iterations = 1)
    #frame_hsv = cv2.dilate(frame_hsv,kernel,iterations = 1)
    


    # Encontramos os contornos na máscara e selecionamos o de maior área
    img_out, contornos, arvore = cv2.findContours(segmentado_cor.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    maior_contorno = None
    maior_contorno_area = 0


    cv2.drawContours(frame, contornos, -1, [255, 0, 255], 5)


    for cnt in contornos:
        area = cv2.contourArea(cnt)


        if area > maior_contorno_area:
            maior_contorno = cnt
            maior_contorno_area = area
            
    h1 = maior_contorno.max()
    h2 = maior_contorno.min()
    h = h1-h2 #altura da caixa na imagem em px
    distancia = (36.5*862.274)/h
    

    
    # Encontramos o centro do contorno fazendo a média de todos seus pontos.
    if not maior_contorno is None :
        cv2.drawContours(frame, [maior_contorno], -1, [0, 0, 255], 5)
        maior_contorno = np.reshape(maior_contorno, (maior_contorno.shape[0], 2))
        media = maior_contorno.mean(axis=0)
        media = media.astype(np.int32)
        cv2.circle(frame, tuple(media), 5, [0, 255, 0])
    else:
        media = (0, 0)

    cv2.imshow('', frame)
    cv2.imshow('imagem in_range', segmentado_cor)
    cv2.waitKey(1)

    centro = (frame.shape[0]//2, frame.shape[1]//2)

    return media, centro, distancia


while(True):
    # Capture frame-by-frame
    #print("Novo frame")
    ret, frame = cap.read()


    img = frame.copy()

    media, centro, distancia = identifica_cor(img)

    #More drawing functions @ http://docs.opencv.org/2.4/modules/core/doc/drawing_functions.html

    # Display the resulting frame
    cv2.imshow('original',frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    #print("No circles were found")
    
    
    print(distancia)
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

# link para o vídeo demonstrativo https://www.youtube.com/watch?v=Ra-Up5y_1f0&app=desktop


93.6696458333
93.9492567164
94.2305419162
93.115387574
91.7580204082
93.9492567164
93.115387574
93.9492567164
91.7580204082
92.0263187135
91.7580204082
93.115387574
92.2961906158
121.517378378
91.7580204082
125.892004
92.0263187135
121.988375969
92.0263187135
117.87640824
110.045458042
105.614097315
117.436570896
108.903117647
104.561465116
115.285717949
103.871290429
104.561465116
104.215235099
103.190167213
104.561465116
99.5981044304
102.517918567
104.910003333
103.529608553
103.190167213
101.525809677
103.190167213
104.215235099
101.525809677
101.854372168
104.215235099
102.185068182
99.9142888889
101.199360129
103.529608553
104.215235099
101.854372168
102.852944444
104.215235099
108.154642612
103.871290429
102.852944444
103.529608553
102.185068182
104.215235099
102.517918567
103.529608553
98.9717012579
102.185068182
107.416385666
103.190167213
102.517918567
102.185068182
104.561465116
103.871290429
101.854372168
100.55271885
102.185068182
102.852944444
103.871290429
102.852944444


104.561465116
103.190167213
105.969700337
105.26087291
105.969700337
107.78425
107.416385666
103.190167213
108.154642612
104.561465116
105.26087291
106.688138983
107.78425
118.319552632
104.561465116
104.215235099
105.614097315
105.614097315
115.7095625
102.517918567
109.66202439
98.9717012579
104.910003333
103.529608553
107.05102381
106.688138983
105.969700337
109.66202439
111.212017668
107.416385666
114.864967153
106.688138983
104.910003333
108.527589655
102.517918567
104.215235099
105.969700337
108.527589655
110.045458042
117.87640824
108.154642612
107.416385666
107.78425
121.988375969
105.969700337
109.281253472
116.56667037
120.586210728
120.586210728
105.26087291
115.7095625
113.620942238
106.327706081
108.154642612
114.447276364
102.852944444
119.215912879
108.154642612
103.190167213
114.032612319
108.154642612
106.327706081
112.403575
114.032612319
108.527589655
104.215235099
113.620942238
108.903117647
118.319552632
114.032612319
115.285717949
119.215912879
114.447276364
108.9

99.5981044304
86.4642884615
86.9419917127
85.9918060109
86.4642884615
86.4642884615
89.1586430595
84.8328867925
85.0621648649
86.4642884615
87.4250027778
87.4250027778
78.6825025
90.439658046
87.4250027778
89.6666695157
87.6685264624
85.5244592391
86.7024820937
86.9419917127
85.7574959128
81.5362720207
84.3780187668
95.0845951662
86.4642884615
88.1596666667
88.6563408451
85.2926856369
89.6666695157
87.6685264624
85.7574959128
98.6614451411
90.439658046
81.9609401042
85.2926856369
83.4827612732
84.1524090909
85.5244592391
85.5244592391
85.7574959128
87.9134106145
101.854372168
85.9918060109
79.0778919598
84.3780187668
84.3780187668
83.4827612732
84.8328867925
90.439658046
82.8236868421
86.4642884615
82.1749373368
86.2274
90.7002910663
100.55271885
90.7002910663
86.4642884615
83.7047898936
87.9134106145
100.232487261
85.0621648649
89.1586430595
87.6685264624
100.55271885
90.9624306358
92.2961906158
101.525809677
99.5981044304
95.372730303
99.9142888889
82.3900549738
101.525809677
84.8328

100.875003205
99.5981044304
99.2839148265
99.2839148265
108.527589655
98.9717012579
98.9717012579
99.2839148265
98.9717012579
100.55271885
99.5981044304
108.903117647
98.9717012579
99.5981044304
99.2839148265
98.9717012579
99.9142888889
102.852944444
104.561465116
102.185068182
99.5981044304
99.2839148265
100.55271885
101.199360129
100.875003205
98.9717012579
98.9717012579
99.2839148265
100.875003205
100.232487261
99.2839148265
99.2839148265
99.2839148265
103.190167213
99.2839148265
100.55271885
99.2839148265
101.525809677
103.529608553
124.399213439
98.0467320872
97.7422391304
111.606386525
104.910003333
98.0467320872
102.185068182
101.525809677
103.190167213
101.854372168
97.4396315789
100.55271885
99.2839148265
97.7422391304
97.4396315789
141.770274775
97.4396315789
108.154642612
101.854372168
98.9717012579
103.190167213
124.892861111
100.875003205
97.1388919753
98.0467320872
100.875003205
98.353128125
98.9717012579
112.403575
103.529608553
104.215235099
98.0467320872
145.036870968


97.1388919753
97.4396315789
116.56667037
105.969700337
98.0467320872
98.9717012579
98.353128125
98.6614451411
112.003562278
99.9142888889
98.6614451411
98.6614451411
98.353128125
98.9717012579
98.6614451411
98.353128125
98.6614451411
99.2839148265
98.353128125
98.353128125
98.9717012579
98.353128125
101.199360129
99.2839148265
100.232487261
102.852944444
113.212233813
104.215235099
152.043483092
98.6614451411
128.987709016
102.852944444
98.6614451411
104.215235099
104.910003333
104.561465116
102.852944444
102.185068182
101.854372168
99.2839148265
101.854372168
103.529608553
104.910003333
98.9717012579
103.190167213
107.78425
101.199360129
103.871290429
99.2839148265
105.26087291
98.6614451411
98.6614451411
98.353128125
98.353128125
100.55271885
99.5981044304
99.9142888889
101.525809677
99.2839148265
98.353128125
98.9717012579
103.190167213
98.353128125
101.199360129
102.517918567
100.875003205
101.525809677
98.353128125
98.353128125
99.9142888889
102.517918567
98.9717012579
106.6881389

97.1388919753
98.353128125
93.115387574
100.232487261
99.2839148265
96.8400030769
98.0467320872
96.8400030769
119.215912879
97.1388919753
97.1388919753
97.7422391304
97.7422391304
102.517918567
104.910003333
99.2839148265
97.1388919753
96.8400030769
96.8400030769
100.232487261
98.353128125
98.353128125
96.8400030769
119.215912879
98.0467320872
96.8400030769
96.8400030769
97.7422391304
98.6614451411
96.8400030769
96.8400030769
102.517918567
97.7422391304
96.5429478528
97.7422391304
96.8400030769
96.8400030769
104.561465116
96.8400030769
97.1388919753
119.669205323
97.4396315789
97.1388919753
96.8400030769
116.56667037
117.87640824
92.8407109145
97.4396315789
106.327706081
102.517918567
97.1388919753
97.1388919753
102.852944444
121.988375969
97.7422391304
121.050003846
98.0467320872
98.0467320872
98.9717012579
100.875003205
130.593365145
121.988375969
96.5429478528
106.688138983
127.42105668
121.988375969
129.518522634
120.125958015
116.56667037
121.988375969
120.586210728
98.6614451411


KeyboardInterrupt: 